# Exploratory Data Analysis
I am using this notebook to learn how gamblers behaviors are similar to those of investors.

## Define Libraries

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive, fixed, IntSlider, HBox, Layout, VBox

# Getting rid of the SettingWithCopyWarning: 
pd.options.mode.chained_assignment = None

## Upload Data

In [ ]:
# Set working directory
path = '/Users/mau/Library/CloudStorage/Dropbox/Mac/Documents/Dissertation/Chapter 2/Data'
os.chdir(path)

# Load data into a DataFrame
dtf = pd.read_parquet("slot_data_sample.parquet")

### Filter Columns and Inspect

In [ ]:


# Select only specific columns
filter = ['playercashableamt', 'wageredamt', 'casino_grosswin', 'playerkey',
       'slotdenominationname', 'slotthemekey']

# Load just specific colums data into a DataFrame
df = pd.read_parquet("slot_data_sample.parquet", columns=filter)

# Print column names
print(df.columns)

# Print general information about the DataFrame
print(df.info())

# Count unique players
print(df['playerkey'].nunique())

## Calculate Foundamental Variables

The following variables were calculated using existing data:
* _player_loss_: how much money each player has lost in each gamble.
* _player_wins_: equals the amount of money they bet plus how much they won.
* _percent_return_: the return in player's bets for each gamble played. 

$$\text{percent return} = (\frac{df[wins] - df[wageredamt]}{df[wageredamt]})*100$$

* _playercashableamt_pct_change_: calculates the rate of change of player's outstanding gambling amount. 

$$\text{playercashableamt \% change} = (\frac{df[playercashableamt_{t+1}] - df[playercashableamt_{t}]}{df[playercashableamt_{t}]})*100$$

In [ ]:
# Crate a new colum that is the inverse of casino_grosswin, named "player_loss"
df["player_loss"] = df["casino_grosswin"] * -1

df['player_wins'] = df['wageredamt'] + df['player_loss']
# Calculate percentage return for each gamble and add it as a new column
df["percent_return"] = (df["player_wins"] - df["wageredamt"]) / df["wageredamt"] * 100

# Calculate the percent rate of change of playerscashableamt per playerkey
df["playercashableamt_pct_change"] = df.groupby("playerkey")["playercashableamt"].pct_change()
# Print the first 5 rows of the DataFrame

# Create a time series variable for each player that starts at 1 and increases by 1 for each row
df["time"] = df.groupby("playerkey").cumcount() + 1

print(df.head())

## Patterns in Return Stream

In this section, I am looking for return stream patterns that are similar to the market returns given to subjects in Saffort et.al 2008 experiemnt. These market returns followed historical returns from the DJIA from 1925 to 1964:

$$\text{\% return DJIA 1925-1964}: [30.0, 0.3, 28.8, 48.2, -17.2, -33.8, -52.7, -23.1, 66.7, 4.1, 38.5, 24.8, -32.8, 28.1, -2.9, -12.7, -15.4, 7.6, 13.8, 12.1, 26.6,\\ -8.1, 2.2, -2.1, 12.9, 17.6, 14.4, 8.4, -3.8, 44.0, 20.8, 2.3, -12.8, 34.0, 16.4, -9.3, 18.7, -10.8, 17.0, 14.0]$$

$$\text{pattern of returns: [1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, 1, -1, 1, -1, -1, -1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1]}$$

### Filter Data
We need to select players who have at least 40 gambles to be able to compare it to the 40 investing periods of Safford's subjects.

In [ ]:
# Create a list of players that appear at least 40 times
players40 = df["playerkey"].value_counts()[df["playerkey"].value_counts() >= 40].index.tolist()
print(players40)
print(len(players40))

# Create a list of players that appear less than 40 times
players40less = df["playerkey"].value_counts()[df["playerkey"].value_counts() < 40].index.tolist()
print(players40less)
print(len(players40less))

# Create a new DataFrame with only the players that appear at least 40 times
df40 = df[df["playerkey"].isin(players40)]
print(df40.shape)

### Patter Recognition
Now that we have filter the data, we can procede to find a patter similar to those of Safford's experiemnt. 

* Define function to look for our desire pattern.
* Define variable _pattern_ and _sign_
* Conduct a hard match.
* Conduct a soft match.

In [ ]:
def match_pattern(df, pattern, initial, window_size):
    col_name = f"match_{initial}_{window_size}"
    size = len(pattern[initial:window_size])
    df.loc[:,col_name] = df["sign"].rolling(size).apply(lambda x: (x == pattern[initial:window_size]).all(), raw=True)
    num_occurrences = df[col_name].sum()
    matching_players = df[df[col_name]== True]["playerkey"].unique().tolist()
    print(f"Pattern:", pattern[initial:window_size])
    print(f"Number of occurrences (pattern size {size}):", num_occurrences)
    print(f"Players matching pattern (pattern size {size}):", matching_players)
    print(f"Count of players (pattern size {size}):", len(matching_players))
    return matching_players


In [ ]:
# Define pattern to search for
pattern = [1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, 1, -1, 1, -1, -1, -1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1]

# Create a new column 'sign' that is 1 if the 'percent_return' is positive -1 else.
df40.loc[:, "sign"] = df40["percent_return"].apply(lambda x: 1 if x > 0 else -1)

In [ ]:
## Hard Match
# Find players that match the pattern exactly
players_match40 = match_pattern(df40, pattern, 0, 40)

In [ ]:
## Soft Match
# Let's see how many times a the first 8 elements of the pattern appear in the data
players_match8 = match_pattern(df40, pattern, 0, 8)

# Let's see how many times a the first 10 elements of the pattern appear in the data
players_match10 = match_pattern(df40, pattern, 0, 10)

# Let's see how many times a the first 20 elements of the pattern appear in the data
players_match20 = match_pattern(df40, pattern, 0, 20)

# Let's see how many times a the pattern from 4 to 11 appears in the data
players_match4_11 = match_pattern(df40, pattern, 4, 11)

# Let's see how many times a the pattern from 4 to 11 appears in the data
players_match4_12 = match_pattern(df40, pattern, 4, 12)

## Slicing DataFrames per Matched Players and Visualizing Outcomes

In this section, we are only inspecting the patterns that have the most subjects. In this section, the code will create a new column called "match_-10_10" that is True for the rows 10 before and 10 after match_0_8 is True. The rolling method with the window size of 21 (10 before + 10 after + 1 current row) will take care of checking the 10 rows before and after the match, and the apply method with the lambda function will check if any of the values in the window is True. Finally, the fillna method will replace any NaN values with False, and the astype method will convert the True/False values to 1/0.


In [ ]:
def filter_match(df, players_match, match_column, rolling_window, fill_value):
    # Create a new DataFrame with only the players that appear in players_match
    df_match_all = df[df["playerkey"].isin(players_match)]

    # Creaete a new column for called match_minus10_0 that is True for the rows around match_column is True
    df_match_all.loc[:, "match_rolling"] = df_match_all[match_column].rolling(window=rolling_window, center=True).apply(lambda x: any(x)).fillna(fill_value).astype(int)
    
    # Slice the DataFrame to only include the rows where match_rolling is True
    df_match_slice = df_match_all[df_match_all["match_rolling"] == True]
    # Return the new DataFrame
    return df_match_all, df_match_slice

In [ ]:
# Create a new DataFrame with only the players that appear in players_match8
df40_match8_10_all, dtf40_match8_10_slice = filter_match(df=df40, players_match=players_match8, match_column="match_0_8", rolling_window=21, fill_value=False)

# Save the DataFrame to a parquet file
df40_match8_10_all.to_parquet("df40_match8_10_all.parquet")
print(df40_match8_10_all.shape)

# Save the DataFrame to a parquet file
dtf40_match8_10_slice.to_parquet("dtf40_match8_10_slice.parquet")
print(dtf40_match8_10_slice.shape)

In [ ]:

# Create a new DataFrame with only the players that appear in players_match4_11
df40_match4_10_all, dtf40_match4_10_slice = filter_match(df=df40, players_match=players_match4_11, match_column="match_4_11", rolling_window=21, fill_value=False)

# Save the DataFrame to a parquet file
df40_match4_10_all.to_parquet("df40_match4_10_all.parquet")
print(df40_match4_10_all.shape)

# Save the DataFrame to a parquet file
dtf40_match4_10_slice.to_parquet("dtf40_match4_10_slice.parquet")
print(dtf40_match4_10_slice.shape)

## Interactive Plots

The following section would be used to explore the data in an interactive way. These plots allow for user interaction, such as zooming, panning, and selecting data points. Users can customize the plot by choosing different variables to plot, adjusting axes ranges, and selecting data subsets. The interactive plots provide a dynamic way to visually explore the data and can reveal patterns or relationships that might not be apparent from static plots alone. By using interactive plots, we can gain a deeper understanding of the data and make more informed decisions during the data analysis process.

In [ ]:
# # Make a list of all the dataframes that are match and slice
# dtf_lists = [dtf40_match8_10_slice, dtf40_match4_10_slice]

# # Create a scatter plot of the players wins for only player with key 3
# def plot_scatters(player_ID, df_index, x="time", y="percent_return", y_2=None):
#     df = dtf_lists[df_index]
#     players = df["playerkey"].unique().tolist()
#     player_df = df[df["playerkey"] == players[player_ID]]
    
#     fig, ax1 = plt.subplots()
#     ax1.set_xlabel(x)
#     ax1.set_ylabel(y, color='royalblue')
#     ax1.scatter(x=player_df[x], y=player_df[y], color='royalblue')
    
#     if y_2 is not None:
#         ax2 = ax1.twinx()
#         ax2.set_ylabel(y_2, color='r')
#         ax2.scatter(x=player_df[x], y=player_df[y_2], color='orangered')
#         ax2.tick_params(axis='y', labelcolor='orangered')

#     ax1.tick_params(axis='y', labelcolor='black')
#     ax1.grid()
#     plt.show()

# # Create widgets for playerkey, df_index, x, y, and y_2
# df_index_widget = widgets.Dropdown(options=[(f"DataFrame {i}", i) for i in range(len(dtf_lists))], value=0)
# x_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value="time")
# y_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value="percent_return")
# y_2_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value=None)


# # Create a function to update the players_widget based on the selected df_index
# def update_players_widget(df_index):
#     df = dtf_lists[df_index]
#     players = df["playerkey"].unique().tolist()
#     player_key_widget.options = [(p, i) for i, p in enumerate(players)]

# # Create a players_widget for the initial df_index value
# initial_df_index = df_index_widget.value
# initial_df = dtf_lists[initial_df_index]
# initial_players = initial_df["playerkey"].unique().tolist()
# player_key_widget = widgets.Dropdown(options=[], value=None)

# # Call update_players_widget with the initial_df_index value to set the options for player_key_widget
# update_players_widget(initial_df_index)

# # Use interact to call the plot_scatters function with the current widget values
# widgets.interact(plot_scatters, player_ID=player_key_widget, df_index=df_index_widget, x=x_widget, y=y_widget, 
#                  y_2=y_2_widget, continuous_update=False)

# # Observe changes to df_index_widget and update the players_widget accordingly
# df_index_widget.observe(lambda change: update_players_widget(change["new"]), names="value")

In [ ]:
# import matplotlib.pyplot as plt
# import ipywidgets as widgets

# # Make a list of all the dataframes that are match and slice
# dtf_lists = [dtf40_match8_10_slice, dtf40_match4_10_slice]

# # Calculate the max and min values for the 'time' column for each DataFrame
# time_max = max([df["time"].max() for df in dtf_lists])
# time_min = min([df["time"].min() for df in dtf_lists])

# print(time_max, time_min)

# # Create a scatter plot of the players wins for only player with key 3
# def plot_scatters(player_ID, df_index, x="time", y="percent_return", y_2=None, x_min=None, x_max=None, show_line=False):
#     df = dtf_lists[df_index]
#     players = df["playerkey"].unique().tolist()
#     player_df = df[df["playerkey"] == players[player_ID]]
    
#     fig, ax1 = plt.subplots()
#     ax1.set_xlabel(x)
#     ax1.set_ylabel(y, color='royalblue')
#     if x_min is not None and x_max is not None:
#         player_df = player_df[(player_df[x] >= x_min) & (player_df[x] <= x_max)]
#     ax1.scatter(x=player_df[x], y=player_df[y], color='royalblue')
    
#     if y_2 is not None:
#         ax2 = ax1.twinx()
#         ax2.set_ylabel(y_2, color='r')
#         if x_min is not None and x_max is not None:
#             player_df = player_df[(player_df[x] >= x_min) & (player_df[x] <= x_max)]
#         ax2.scatter(x=player_df[x], y=player_df[y_2], color='orangered', marker='s')
#         ax2.tick_params(axis='y', labelcolor='orangered')
#         # Add a line to the plot if show_line is True
#         if show_line:
#             ax2.plot(player_df[x], player_df[y_2], color='black', linewidth=0.8, linestyle='--')
    
#     if show_line:
#         ax1.plot(player_df[x], player_df[y], color='black', linewidth=0.8)
        
        
#     ax1.tick_params(axis='y', labelcolor='black')
#     ax1.grid()
#     plt.title(f"Player {players[player_ID]}")
#     plt.show()

# # Create widgets for playerkey, df_index, x, y, y_2, x_min, and x_max
# df_index_widget = widgets.Dropdown(options=[(f"DataFrame {i}", i) for i in range(len(dtf_lists))], value=0)
# x_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value="time")
# y_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value="percent_return")
# y_2_widget = widgets.Dropdown(options=[None]+list(dtf_lists[0].columns), value=None)
# x_min_widget = widgets.FloatText(description="x_min", value=time_min)
# x_max_widget = widgets.FloatText(description="x_max", value=time_max)
# show_line_widget = widgets.Checkbox(description='Show line', value=False)


# # Create a function to update the players_widget based on the selected df_index
# def update_players_widget(df_index):
#     df = dtf_lists[df_index]
#     players = df["playerkey"].unique().tolist()
#     player_key_widget.options = [(p, i) for i, p in enumerate(players)]

# # Create a players_widget for the initial df_index value
# initial_df_index = df_index_widget.value
# initial_df = dtf_lists[initial_df_index]
# initial_players = initial_df["playerkey"].unique().tolist()
# player_key_widget = widgets.Dropdown(options=[], value=None)

# # Call update_players_widget with the initial_df_index value to set the options for player_key_widget
# update_players_widget(initial_df_index)

# # Use interact to call the plot_scatters function with the current widget values
# widgets.interact(plot_scatters, player_ID=player_key_widget, df_index=df_index_widget,
#                  x=x_widget, y=y_widget, y_2=y_2_widget, x_min=x_min_widget, x_max=x_max_widget,
#                  show_line=show_line_widget)

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Make a list of all the dataframes that are match and slice
dtf_lists = [dtf40_match8_10_slice, dtf40_match4_10_slice]

# Calculate the max and min values for the 'time' column for each DataFrame
time_max = max([df["time"].max() for df in dtf_lists])
time_min = min([df["time"].min() for df in dtf_lists])

print(time_max, time_min)

# Create a scatter plot of the players wins for only player with key 3
def plot_scatters(player_ID, df_index, x="time", y="percent_return", y_2=None, x_min=None, x_max=None, show_line=False, shade_area=False):
    df = dtf_lists[df_index]
    players = df["playerkey"].unique().tolist()
    player_df = df[df["playerkey"] == players[player_ID]]
    
    fig, ax1 = plt.subplots()
    ax1.set_xlabel(x)
    ax1.set_ylabel(y, color='royalblue')
    if x_min is not None and x_max is not None:
        player_df = player_df[(player_df[x] >= x_min) & (player_df[x] <= x_max)]
    ax1.scatter(x=player_df[x], y=player_df[y], color='royalblue')
    
    if y_2 is not None:
        ax2 = ax1.twinx()
        ax2.set_ylabel(y_2, color='r')
        if x_min is not None and x_max is not None:
            player_df = player_df[(player_df[x] >= x_min) & (player_df[x] <= x_max)]
        ax2.scatter(x=player_df[x], y=player_df[y_2], color='orangered', marker='s')
        ax2.tick_params(axis='y', labelcolor='orangered')
        # Add a line to the plot if show_line is True
        if show_line:
            ax2.plot(player_df[x], player_df[y_2], color='black', linewidth=0.8, linestyle='--')
        if shade_area:
            ax2.fill_between(player_df[x], player_df[y_2], color='lightcoral', alpha=0.5)
    
    if show_line:
        ax1.plot(player_df[x], player_df[y], color='black', linewidth=0.8)
        
    if shade_area:
        ax1.fill_between(player_df[x], player_df[y], color='lightblue', alpha=0.5)

    ax1.tick_params(axis='y', labelcolor='black')
    ax1.grid()
    plt.title(f"Player {players[player_ID]}")
    plt.show()

# Create widgets for playerkey, df_index, x, y, y_2, x_min, and x_max
df_index_widget = widgets.Dropdown(options=[(f"DataFrame {i}", i) for i in range(len(dtf_lists))], value=0)
x_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value="time")
y_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value="percent_return")
y_2_widget = widgets.Dropdown(options=[None]+list(dtf_lists[0].columns), value=None)
x_min_widget = widgets.FloatText(description="x_min", value=time_min)
x_max_widget = widgets.FloatText(description="x_max", value=time_max)
show_line_widget = widgets.Checkbox(description='Show line', value=False)
shade_area_widget = widgets.Checkbox(description='Shade area', value=False)

# Create a function to update the players_widget based on the selected df_index
def update_players_widget(df_index):
    df = dtf_lists[df_index]
    players = df["playerkey"].unique().tolist()
    player_key_widget.options = [(p, i) for i, p in enumerate(players)]

# Create a players_widget for the initial df_index value
initial_df_index = df_index_widget.value
initial_df = dtf_lists[initial_df_index]
initial_players = initial_df["playerkey"].unique().tolist()
player_key_widget = widgets.Dropdown(options=[], value=None)

# Call update_players_widget with the initial_df_index value to set the options for player_key_widget
update_players_widget(initial_df_index)

widgets.interact(plot_scatters, player_ID=player_key_widget, df_index=df_index_widget,
                 x=x_widget, y=y_widget, y_2=y_2_widget, x_min=x_min_widget, x_max=x_max_widget,
                 show_line=show_line_widget, shade_area=shade_area_widget)